# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [120]:
# use normalization to create tables from nested element
json_normalize(data)

,counties,info.governor,shortname,state
0,"[{'name': 'Dade', 'population': 12345}, {'name...",Rick Scott,FL,Florida
1,"[{'name': 'Summit', 'population': 1234}, {'nam...",John Kasich,OH,Ohio


In [11]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [12]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{'_id': {'$oid': '52b213b38594d8a2be17c780'},
  'approvalfy': 1999,
  'board_approval_month': 'November',
  'boardapprovaldate': '2013-11-12T00:00:00Z',
  'borrower': 'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  'closingdate': '2018-07-07T00:00:00Z',
  'country_namecode': 'Federal Democratic Republic of Ethiopia!$!ET',
  'countrycode': 'ET',
  'countryname': 'Federal Democratic Republic of Ethiopia',
  'countryshortname': 'Ethiopia',
  'docty': 'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  'envassesmentcategorycode': 'C',
  'grantamt': 0,
  'ibrdcommamt': 0,
  'id': 'P129828',
  'idacommamt': 130000000,
  'impagency': 'MINISTRY OF EDUCATION',
  'lendinginstr': 'Investment Project Financing',
  'lendinginstrtype': 'IN',
  'lendprojectcost': 550000000,
  'majorsector_percent': [{'Name': 'Education', 'Percent': 46},
   {'Name': 'Education', 'Percent': 26},
   {'Name': 'Public Administration, Law, and Justice', 'Percent': 16},
   {'Name': 'Educatio

In [13]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [49]:
#1. Find the 10 countries with most projects
full_json_df = pd.read_json('data/world_bank_projects.json')
# My initial script produced Africa as one of the countries. Since Africa is not a country but a continent, 
# I first filter out countryname Africa
countries = full_json_df[full_json_df.countryname!="Africa"]['countryname']
# next, group the countries by name, count, and sort in descending order
countries_count = countries.groupby(countries).count()
countries_count = countries_count.sort_values(ascending=False)
# top 10 countries by the number of projects are printed
print(countries_count.head(10))

countryname
Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Republic of Mozambique             11
Burkina Faso                        9
Name: countryname, dtype: int64


In [122]:
#2. Find the top 10 major project themes (using column 'mjtheme_namecode')   
def norm_data(row):
    print(row)
    return json_normalize(row)
    
#project_df = full_json_df['mjtheme_namecode'].map(json_normalize)
project_df = full_json_df['mjtheme_namecode'].apply(norm_data)
print(project_df)
#project_df_pivoted = pd.pivot_table(project_df, values=None, index=None, columns=['name', 'code'], aggfunc='count')
#project_df = json_normalize(full_json_df['mjtheme_namecode'])
#print(project_df_pivoted)

print(full_json_df['mjtheme_namecode'].head())
#print(major_themes_df.head())

#major_themes_df = pd.read_json(full_json_df['mjtheme_namecode'])
 
 #json_normalize(full_json_df['mjtheme_namecode'])
#print(major_themes_df.head())
#project_themes = full_json_df.groupby('mjtheme_namecode').count()['countryname']
#project_themes = project_themes.sort_values(ascending=False)
#print(project_themes.head())

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]
[{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]
[{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]
[{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]
[{'code': '5', 'name': 'Trade and integration'}, {'code': '4', 'name': 'Financial and private sector development'}]
[{'code': '6', 'name': 'Social protection and risk management'}, {'code': '6', 'name': ''}]
[{'code': '2', 'name': 'Public sector governance'}, {'code': '4', 'name': 'Financial and private sector development'}]
[{'code': '11', 'name': 'Environment and natural resources management'}, {'code': '8', 'name': ''}]
[{'code': '10', 'name': 'Rural development'}

[{'code': '11', 'name': 'Environment and natural resources management'}, {'code': '11', 'name': ''}]
[{'code': '9', 'name': 'Urban development'}, {'code': '4', 'name': 'Financial and private sector development'}, {'code': '11', 'name': 'Environment and natural resources management'}]
[{'code': '6', 'name': 'Social protection and risk management'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '4', 'name': 'Financial and private sector development'}, {'code': '1', 'name': 'Economic management'}]
[{'code': '1', 'name': 'Economic management'}, {'code': '4', 'name': 'Financial and private sector development'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '10', 'name': 'Rural development'}, {'code': '1', 'name': 'Economic management'}]
[{'code': '8', 'name': 'Human development'}, {'code': '9', 'name': ''}]
[{'code': '8', 'name': 'Human development'}, {'code': '2', 'name': ''}]
[{'code': '11', 'name': 'Environment and natural resources management'}, {'code': '11'

[{'code': '2', 'name': 'Public sector governance'}, {'code': '5', 'name': ''}]
[{'code': '6', 'name': 'Social protection and risk management'}, {'code': '10', 'name': ''}]
[{'code': '6', 'name': 'Social protection and risk management'}, {'code': '6', 'name': ''}]
[{'code': '4', 'name': 'Financial and private sector development'}, {'code': '4', 'name': 'Financial and private sector development'}]
[{'code': '4', 'name': 'Financial and private sector development'}, {'code': '8', 'name': ''}]
[{'code': '10', 'name': 'Rural development'}, {'code': '10', 'name': 'Rural development'}]
[{'code': '10', 'name': 'Rural development'}, {'code': '8', 'name': ''}]
[{'code': '6', 'name': 'Social protection and risk management'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '6', 'name': 'Social protection and risk management'}, {'code': '6', 'name': 'Social protection and risk management'}]
[{'code': '5', 'name': 'Trade and integration'}, {'code': '10', 'name': 'Rural development'}, {

Name: mjtheme_namecode, Length: 500, dtype: object
0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object


In [ ]:
#3. In 2. above you will notice that some entries have only the code and the name is missing. 
# Create a dataframe with the missing names filled in.